In [1]:
import pandas as pd

hr = pd.read_excel("data/raw/heart_rate.xlsx")

In [2]:
hr.head()

,type,sourceName,sourceVersion,device,unit,creationDate,startDate,endDate,value,MetadataEntry
0,HKQuantityTypeIdentifierHeartRate,Hiro’s Apple Watch,11.0.1,"<<HKDevice: 0x6ca0084e0>, name:Apple Watch, ma...",count/min,2024-11-09 05:59:13 +0530,2024-11-09 05:56:05 +0530,2024-11-09 05:56:05 +0530,84.0,NaN
1,HKQuantityTypeIdentifierHeartRate,Hiro’s Apple Watch,11.0.1,"<<HKDevice: 0x6ca0084e0>, name:Apple Watch, ma...",count/min,2024-11-09 06:02:16 +0530,2024-11-09 05:57:47 +0530,2024-11-09 05:57:47 +0530,77.0,NaN
2,HKQuantityTypeIdentifierHeartRate,Hiro’s Apple Watch,11.0.1,"<<HKDevice: 0x6ca0084e0>, name:Apple Watch, ma...",count/min,2024-11-09 06:10:13 +0530,2024-11-09 06:05:50 +0530,2024-11-09 06:05:50 +0530,73.0,NaN
3,HKQuantityTypeIdentifierHeartRate,Hiro’s Apple Watch,11.0.1,"<<HKDevice: 0x6ca0084e0>, name:Apple Watch, ma...",count/min,2024-11-09 06:11:55 +0530,2024-11-09 06:07:50 +0530,2024-11-09 06:07:50 +0530,74.0,NaN
4,HKQuantityTypeIdentifierHeartRate,Hiro’s Apple Watch,11.0.1,"<<HKDevice: 0x6ca0084e0>, name:Apple Watch, ma...",count/min,2024-11-09 06:18:10 +0530,2024-11-09 06:11:35 +0530,2024-11-09 06:11:35 +0530,79.0,NaN


In [13]:
hr.shape

(208255, 10)

In [4]:
hr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 208255 entries, 0 to 208254
Data columns (total 10 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   type           208255 non-null  object 
 1   sourceName     208255 non-null  object 
 2   sourceVersion  208255 non-null  object 
 3   device         172636 non-null  object 
 4   unit           208255 non-null  object 
 5   creationDate   208255 non-null  object 
 6   startDate      208255 non-null  object 
 7   endDate        208255 non-null  object 
 8   value          208255 non-null  float64
 9   MetadataEntry  0 non-null       float64
dtypes: float64(2), object(8)
memory usage: 15.9+ MB


In [6]:
#missing data check
hr.isna().sum()

type                  0
sourceName            0
sourceVersion         0
device            35619
unit                  0
creationDate          0
startDate             0
endDate               0
value                 0
MetadataEntry    208255
dtype: int64

In [9]:
(hr.isna().mean()*100).round(2)

type               0.0
sourceName         0.0
sourceVersion      0.0
device            17.1
unit               0.0
creationDate       0.0
startDate          0.0
endDate            0.0
value              0.0
MetadataEntry    100.0
dtype: float64

In [10]:
hr["value"].describe()

count    208255.000000
mean        100.353263
std          25.458927
min           0.000000
25%          79.000000
50%         103.000000
75%         120.000000
max         203.000000
Name: value, dtype: float64

In [19]:
#count clearly invalid heart rate values
invalid_hr_count = hr[(hr["value"] <= 20) | (hr["value"] >= 250)].shape[0]

In [20]:
invalid_hr_count

15

In [21]:
(hr["value"]==0).sum()

np.int64(15)

In [22]:
hr.loc[hr["value"] == 0, ["startDate", "endDate"]].head(10)

,startDate,endDate
176628,2025-10-27 18:26:34 +0530,2025-10-27 18:26:34 +0530
177118,2025-10-27 19:16:51 +0530,2025-10-27 19:16:51 +0530
178765,2025-11-03 19:36:11 +0530,2025-11-03 19:36:11 +0530
179938,2025-11-05 20:28:35 +0530,2025-11-05 20:28:35 +0530
180356,2025-11-07 19:50:02 +0530,2025-11-07 19:50:02 +0530
180542,2025-11-07 19:58:42 +0530,2025-11-07 19:58:42 +0530
180543,2025-11-07 19:58:54 +0530,2025-11-07 19:58:54 +0530
182625,2025-11-07 19:06:39 +0530,2025-11-07 19:06:39 +0530
190230,2025-11-18 19:01:07 +0530,2025-11-18 19:01:07 +0530
191478,2025-11-14 19:40:39 +0530,2025-11-14 19:40:39 +0530


In [28]:
#convert startDate into datetime
hr["startDate"] = pd.to_datetime(hr["startDate"])

In [29]:
hr["startDate"].dtype

datetime64[ns, UTC+05:30]

In [30]:
#sort by startDate
hr_sorted = hr.sort_values("startDate")

In [33]:
time_diff = hr_sorted["startDate"].diff()
time_diff.value_counts().head(10)

#shows error because startDate here is a str datatype

startDate
0 days 00:00:05    24761
0 days 00:00:01    22265
0 days 00:00:04    13392
0 days 00:00:02    13195
0 days 00:00:03    12180
0 days 00:00:06    10183
0 days 00:00:09     9343
0 days 00:00:07     7630
0 days 00:00:08     6854
0 days 00:01:00     5255
Name: count, dtype: int64

In [36]:
time_diff.dt.total_seconds().mean() #the value of mean is more due to long gaps

np.float64(552.1464173557291)

In [37]:
time_diff.dt.total_seconds().median()

7.0

In [38]:
#create a working copy
hr_clean = hr.copy()

In [39]:
hr_clean

,type,sourceName,sourceVersion,device,unit,creationDate,startDate,endDate,value,MetadataEntry
0,HKQuantityTypeIdentifierHeartRate,Hiro’s Apple Watch,11.0.1,"<<HKDevice: 0x6ca0084e0>, name:Apple Watch, ma...",count/min,2024-11-09 05:59:13 +0530,2024-11-09 05:56:05+05:30,2024-11-09 05:56:05 +0530,84.0,NaN
1,HKQuantityTypeIdentifierHeartRate,Hiro’s Apple Watch,11.0.1,"<<HKDevice: 0x6ca0084e0>, name:Apple Watch, ma...",count/min,2024-11-09 06:02:16 +0530,2024-11-09 05:57:47+05:30,2024-11-09 05:57:47 +0530,77.0,NaN
2,HKQuantityTypeIdentifierHeartRate,Hiro’s Apple Watch,11.0.1,"<<HKDevice: 0x6ca0084e0>, name:Apple Watch, ma...",count/min,2024-11-09 06:10:13 +0530,2024-11-09 06:05:50+05:30,2024-11-09 06:05:50 +0530,73.0,NaN
3,HKQuantityTypeIdentifierHeartRate,Hiro’s Apple Watch,11.0.1,"<<HKDevice: 0x6ca0084e0>, name:Apple Watch, ma...",count/min,2024-11-09 06:11:55 +0530,2024-11-09 06:07:50+05:30,2024-11-09 06:07:50 +0530,74.0,NaN
4,HKQuantityTypeIdentifierHeartRate,Hiro’s Apple Watch,11.0.1,"<<HKDevice: 0x6ca0084e0>, name:Apple Watch, ma...",count/min,2024-11-09 06:18:10 +0530,2024-11-09 06:11:35+05:30,2024-11-09 06:11:35 +0530,79.0,NaN
...,...,...,...,...,...,...,...,...,...,...
208250,HKQuantityTypeIdentifierHeartRate,Zepp,202511201645,NaN,count/min,2025-12-03 07:19:37 +0530,2025-12-02 21:36:51+05:30,2025-12-02 21:36:51 +0530,107.0,NaN
208251,HKQuantityTypeIdentifierHeartRate,Zepp,202511201645,NaN,count/min,2025-12-03 07:19:37 +0530,2025-12-02 21:36:56+05:30,2025-12-02 21:36:56 +0530,107.0,NaN
208252,HKQuantityTypeIdentifierHeartRate,Zepp,202511201645,NaN,count/min,2025-12-03 07:19:37 +0530,2025-12-02 21:37:02+05:30,2025-12-02 21:37:02 +0530,107.0,NaN
208253,HKQuantityTypeIdentifierHeartRate,Zepp,202511201645,NaN,count/min,2025-12-03 07:19:37 +0530,2025-12-02 21:37:04+05:30,2025-12-02 21:37:04 +0530,106.0,NaN


In [41]:
hr_clean["invalid_flag"] = (( hr_clean["value"] <= 20)| (hr_clean["value"] >=250)).astype(int)

In [43]:
#adding invalid flags for hr<=20 or hr>=250
hr_clean["invalid_flag"].value_counts()

invalid_flag
0    208240
1        15
Name: count, dtype: int64

In [44]:
hr_clean

,type,sourceName,sourceVersion,device,unit,creationDate,startDate,endDate,value,MetadataEntry,invalid_flag
0,HKQuantityTypeIdentifierHeartRate,Hiro’s Apple Watch,11.0.1,"<<HKDevice: 0x6ca0084e0>, name:Apple Watch, ma...",count/min,2024-11-09 05:59:13 +0530,2024-11-09 05:56:05+05:30,2024-11-09 05:56:05 +0530,84.0,NaN,0
1,HKQuantityTypeIdentifierHeartRate,Hiro’s Apple Watch,11.0.1,"<<HKDevice: 0x6ca0084e0>, name:Apple Watch, ma...",count/min,2024-11-09 06:02:16 +0530,2024-11-09 05:57:47+05:30,2024-11-09 05:57:47 +0530,77.0,NaN,0
2,HKQuantityTypeIdentifierHeartRate,Hiro’s Apple Watch,11.0.1,"<<HKDevice: 0x6ca0084e0>, name:Apple Watch, ma...",count/min,2024-11-09 06:10:13 +0530,2024-11-09 06:05:50+05:30,2024-11-09 06:05:50 +0530,73.0,NaN,0
3,HKQuantityTypeIdentifierHeartRate,Hiro’s Apple Watch,11.0.1,"<<HKDevice: 0x6ca0084e0>, name:Apple Watch, ma...",count/min,2024-11-09 06:11:55 +0530,2024-11-09 06:07:50+05:30,2024-11-09 06:07:50 +0530,74.0,NaN,0
4,HKQuantityTypeIdentifierHeartRate,Hiro’s Apple Watch,11.0.1,"<<HKDevice: 0x6ca0084e0>, name:Apple Watch, ma...",count/min,2024-11-09 06:18:10 +0530,2024-11-09 06:11:35+05:30,2024-11-09 06:11:35 +0530,79.0,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...
208250,HKQuantityTypeIdentifierHeartRate,Zepp,202511201645,NaN,count/min,2025-12-03 07:19:37 +0530,2025-12-02 21:36:51+05:30,2025-12-02 21:36:51 +0530,107.0,NaN,0
208251,HKQuantityTypeIdentifierHeartRate,Zepp,202511201645,NaN,count/min,2025-12-03 07:19:37 +0530,2025-12-02 21:36:56+05:30,2025-12-02 21:36:56 +0530,107.0,NaN,0
208252,HKQuantityTypeIdentifierHeartRate,Zepp,202511201645,NaN,count/min,2025-12-03 07:19:37 +0530,2025-12-02 21:37:02+05:30,2025-12-02 21:37:02 +0530,107.0,NaN,0
208253,HKQuantityTypeIdentifierHeartRate,Zepp,202511201645,NaN,count/min,2025-12-03 07:19:37 +0530,2025-12-02 21:37:04+05:30,2025-12-02 21:37:04 +0530,106.0,NaN,0


In [46]:
hr_clean["zero_flag"] = (hr_clean["value"]==0).astype(int)

In [49]:
hr_clean["zero_flag"].value_counts()

zero_flag
0    208240
1        15
Name: count, dtype: int64

In [52]:
#converting 0 to NaN .. as it is impossible
import numpy as np

In [53]:
hr_clean.loc[hr_clean["zero_flag"]==1 , "value"] = np.nan

In [54]:
hr_clean["value"].isna().sum()

np.int64(15)

In [55]:
hr_clean["value"].describe()

count    208240.000000
mean        100.360491
std          25.445592
min          40.000000
25%          79.000000
50%         103.000000
75%         120.000000
max         203.000000
Name: value, dtype: float64

In [57]:
hr_clean = hr_clean.drop(columns=["MetadataEntry"])

In [58]:
hr_clean

,type,sourceName,sourceVersion,device,unit,creationDate,startDate,endDate,value,invalid_flag,zero_flag
0,HKQuantityTypeIdentifierHeartRate,Hiro’s Apple Watch,11.0.1,"<<HKDevice: 0x6ca0084e0>, name:Apple Watch, ma...",count/min,2024-11-09 05:59:13 +0530,2024-11-09 05:56:05+05:30,2024-11-09 05:56:05 +0530,84.0,0,0
1,HKQuantityTypeIdentifierHeartRate,Hiro’s Apple Watch,11.0.1,"<<HKDevice: 0x6ca0084e0>, name:Apple Watch, ma...",count/min,2024-11-09 06:02:16 +0530,2024-11-09 05:57:47+05:30,2024-11-09 05:57:47 +0530,77.0,0,0
2,HKQuantityTypeIdentifierHeartRate,Hiro’s Apple Watch,11.0.1,"<<HKDevice: 0x6ca0084e0>, name:Apple Watch, ma...",count/min,2024-11-09 06:10:13 +0530,2024-11-09 06:05:50+05:30,2024-11-09 06:05:50 +0530,73.0,0,0
3,HKQuantityTypeIdentifierHeartRate,Hiro’s Apple Watch,11.0.1,"<<HKDevice: 0x6ca0084e0>, name:Apple Watch, ma...",count/min,2024-11-09 06:11:55 +0530,2024-11-09 06:07:50+05:30,2024-11-09 06:07:50 +0530,74.0,0,0
4,HKQuantityTypeIdentifierHeartRate,Hiro’s Apple Watch,11.0.1,"<<HKDevice: 0x6ca0084e0>, name:Apple Watch, ma...",count/min,2024-11-09 06:18:10 +0530,2024-11-09 06:11:35+05:30,2024-11-09 06:11:35 +0530,79.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
208250,HKQuantityTypeIdentifierHeartRate,Zepp,202511201645,NaN,count/min,2025-12-03 07:19:37 +0530,2025-12-02 21:36:51+05:30,2025-12-02 21:36:51 +0530,107.0,0,0
208251,HKQuantityTypeIdentifierHeartRate,Zepp,202511201645,NaN,count/min,2025-12-03 07:19:37 +0530,2025-12-02 21:36:56+05:30,2025-12-02 21:36:56 +0530,107.0,0,0
208252,HKQuantityTypeIdentifierHeartRate,Zepp,202511201645,NaN,count/min,2025-12-03 07:19:37 +0530,2025-12-02 21:37:02+05:30,2025-12-02 21:37:02 +0530,107.0,0,0
208253,HKQuantityTypeIdentifierHeartRate,Zepp,202511201645,NaN,count/min,2025-12-03 07:19:37 +0530,2025-12-02 21:37:04+05:30,2025-12-02 21:37:04 +0530,106.0,0,0


In [59]:
hr_clean["device"].nunique(dropna="False")

333

In [60]:
hr_clean["device"].value_counts(dropna = "False").head(10)

device
<<HKDevice: 0x6ca008b40>, name:Apple Watch, manufacturer:Apple Inc., model:Watch, hardware:Watch6,11, software:10.3, creation date:2024-01-25 10:47:32 +0000>    1801
<<HKDevice: 0x6ca00a520>, name:Apple Watch, manufacturer:Apple Inc., model:Watch, hardware:Watch6,11, software:10.3, creation date:2024-01-25 10:47:32 +0000>    1800
<<HKDevice: 0x6ca008360>, name:Apple Watch, manufacturer:Apple Inc., model:Watch, hardware:Watch6,11, software:10.3, creation date:2024-01-25 10:47:32 +0000>    1800
<<HKDevice: 0x6ca00ae80>, name:Apple Watch, manufacturer:Apple Inc., model:Watch, hardware:Watch6,11, software:10.3, creation date:2024-01-25 10:47:32 +0000>    1800
<<HKDevice: 0x6ca00b900>, name:Apple Watch, manufacturer:Apple Inc., model:Watch, hardware:Watch6,11, software:10.3, creation date:2024-01-25 10:47:32 +0000>    1800
<<HKDevice: 0x6ca009b00>, name:Apple Watch, manufacturer:Apple Inc., model:Watch, hardware:Watch6,11, software:10.3, creation date:2024-01-25 10:47:32 +0000>    18

In [61]:
hr_clean = hr_clean.drop(columns=["device"])

In [62]:
hr_clean.columns

Index(['type', 'sourceName', 'sourceVersion', 'unit', 'creationDate',
       'startDate', 'endDate', 'value', 'invalid_flag', 'zero_flag'],
      dtype='object')

In [63]:
hr_clean

,type,sourceName,sourceVersion,unit,creationDate,startDate,endDate,value,invalid_flag,zero_flag
0,HKQuantityTypeIdentifierHeartRate,Hiro’s Apple Watch,11.0.1,count/min,2024-11-09 05:59:13 +0530,2024-11-09 05:56:05+05:30,2024-11-09 05:56:05 +0530,84.0,0,0
1,HKQuantityTypeIdentifierHeartRate,Hiro’s Apple Watch,11.0.1,count/min,2024-11-09 06:02:16 +0530,2024-11-09 05:57:47+05:30,2024-11-09 05:57:47 +0530,77.0,0,0
2,HKQuantityTypeIdentifierHeartRate,Hiro’s Apple Watch,11.0.1,count/min,2024-11-09 06:10:13 +0530,2024-11-09 06:05:50+05:30,2024-11-09 06:05:50 +0530,73.0,0,0
3,HKQuantityTypeIdentifierHeartRate,Hiro’s Apple Watch,11.0.1,count/min,2024-11-09 06:11:55 +0530,2024-11-09 06:07:50+05:30,2024-11-09 06:07:50 +0530,74.0,0,0
4,HKQuantityTypeIdentifierHeartRate,Hiro’s Apple Watch,11.0.1,count/min,2024-11-09 06:18:10 +0530,2024-11-09 06:11:35+05:30,2024-11-09 06:11:35 +0530,79.0,0,0
...,...,...,...,...,...,...,...,...,...,...
208250,HKQuantityTypeIdentifierHeartRate,Zepp,202511201645,count/min,2025-12-03 07:19:37 +0530,2025-12-02 21:36:51+05:30,2025-12-02 21:36:51 +0530,107.0,0,0
208251,HKQuantityTypeIdentifierHeartRate,Zepp,202511201645,count/min,2025-12-03 07:19:37 +0530,2025-12-02 21:36:56+05:30,2025-12-02 21:36:56 +0530,107.0,0,0
208252,HKQuantityTypeIdentifierHeartRate,Zepp,202511201645,count/min,2025-12-03 07:19:37 +0530,2025-12-02 21:37:02+05:30,2025-12-02 21:37:02 +0530,107.0,0,0
208253,HKQuantityTypeIdentifierHeartRate,Zepp,202511201645,count/min,2025-12-03 07:19:37 +0530,2025-12-02 21:37:04+05:30,2025-12-02 21:37:04 +0530,106.0,0,0


In [64]:
hr_clean["type"].value_counts()

type
HKQuantityTypeIdentifierHeartRate    208255
Name: count, dtype: int64

In [65]:
hr_clean["metric"] = "heart_rate"

In [66]:
hr_clean

,type,sourceName,sourceVersion,unit,creationDate,startDate,endDate,value,invalid_flag,zero_flag,metric
0,HKQuantityTypeIdentifierHeartRate,Hiro’s Apple Watch,11.0.1,count/min,2024-11-09 05:59:13 +0530,2024-11-09 05:56:05+05:30,2024-11-09 05:56:05 +0530,84.0,0,0,heart_rate
1,HKQuantityTypeIdentifierHeartRate,Hiro’s Apple Watch,11.0.1,count/min,2024-11-09 06:02:16 +0530,2024-11-09 05:57:47+05:30,2024-11-09 05:57:47 +0530,77.0,0,0,heart_rate
2,HKQuantityTypeIdentifierHeartRate,Hiro’s Apple Watch,11.0.1,count/min,2024-11-09 06:10:13 +0530,2024-11-09 06:05:50+05:30,2024-11-09 06:05:50 +0530,73.0,0,0,heart_rate
3,HKQuantityTypeIdentifierHeartRate,Hiro’s Apple Watch,11.0.1,count/min,2024-11-09 06:11:55 +0530,2024-11-09 06:07:50+05:30,2024-11-09 06:07:50 +0530,74.0,0,0,heart_rate
4,HKQuantityTypeIdentifierHeartRate,Hiro’s Apple Watch,11.0.1,count/min,2024-11-09 06:18:10 +0530,2024-11-09 06:11:35+05:30,2024-11-09 06:11:35 +0530,79.0,0,0,heart_rate
...,...,...,...,...,...,...,...,...,...,...,...
208250,HKQuantityTypeIdentifierHeartRate,Zepp,202511201645,count/min,2025-12-03 07:19:37 +0530,2025-12-02 21:36:51+05:30,2025-12-02 21:36:51 +0530,107.0,0,0,heart_rate
208251,HKQuantityTypeIdentifierHeartRate,Zepp,202511201645,count/min,2025-12-03 07:19:37 +0530,2025-12-02 21:36:56+05:30,2025-12-02 21:36:56 +0530,107.0,0,0,heart_rate
208252,HKQuantityTypeIdentifierHeartRate,Zepp,202511201645,count/min,2025-12-03 07:19:37 +0530,2025-12-02 21:37:02+05:30,2025-12-02 21:37:02 +0530,107.0,0,0,heart_rate
208253,HKQuantityTypeIdentifierHeartRate,Zepp,202511201645,count/min,2025-12-03 07:19:37 +0530,2025-12-02 21:37:04+05:30,2025-12-02 21:37:04 +0530,106.0,0,0,heart_rate


In [67]:
hr_clean = hr_clean.drop(columns=["type"])

In [68]:
hr_clean

,sourceName,sourceVersion,unit,creationDate,startDate,endDate,value,invalid_flag,zero_flag,metric
0,Hiro’s Apple Watch,11.0.1,count/min,2024-11-09 05:59:13 +0530,2024-11-09 05:56:05+05:30,2024-11-09 05:56:05 +0530,84.0,0,0,heart_rate
1,Hiro’s Apple Watch,11.0.1,count/min,2024-11-09 06:02:16 +0530,2024-11-09 05:57:47+05:30,2024-11-09 05:57:47 +0530,77.0,0,0,heart_rate
2,Hiro’s Apple Watch,11.0.1,count/min,2024-11-09 06:10:13 +0530,2024-11-09 06:05:50+05:30,2024-11-09 06:05:50 +0530,73.0,0,0,heart_rate
3,Hiro’s Apple Watch,11.0.1,count/min,2024-11-09 06:11:55 +0530,2024-11-09 06:07:50+05:30,2024-11-09 06:07:50 +0530,74.0,0,0,heart_rate
4,Hiro’s Apple Watch,11.0.1,count/min,2024-11-09 06:18:10 +0530,2024-11-09 06:11:35+05:30,2024-11-09 06:11:35 +0530,79.0,0,0,heart_rate
...,...,...,...,...,...,...,...,...,...,...
208250,Zepp,202511201645,count/min,2025-12-03 07:19:37 +0530,2025-12-02 21:36:51+05:30,2025-12-02 21:36:51 +0530,107.0,0,0,heart_rate
208251,Zepp,202511201645,count/min,2025-12-03 07:19:37 +0530,2025-12-02 21:36:56+05:30,2025-12-02 21:36:56 +0530,107.0,0,0,heart_rate
208252,Zepp,202511201645,count/min,2025-12-03 07:19:37 +0530,2025-12-02 21:37:02+05:30,2025-12-02 21:37:02 +0530,107.0,0,0,heart_rate
208253,Zepp,202511201645,count/min,2025-12-03 07:19:37 +0530,2025-12-02 21:37:04+05:30,2025-12-02 21:37:04 +0530,106.0,0,0,heart_rate


In [70]:
hr_clean["sourceName"].value_counts()

sourceName
Hiro’s Apple Watch    172636
Zepp                   35452
Power Sync               166
Fitdays                    1
Name: count, dtype: int64

In [78]:
display(hr_clean.loc[ hr_clean.sourceName == "FitDays"])

,sourceName,sourceVersion,unit,creationDate,startDate,endDate,value,invalid_flag,zero_flag,metric


In [79]:
hr_clean["sourceName"].value_counts()

sourceName
Hiro’s Apple Watch    172636
Zepp                   35452
Power Sync               166
Fitdays                    1
Name: count, dtype: int64

In [81]:
display(hr_clean.loc[ (hr_clean.sourceName == "FitDays") | (hr_clean.sourceName == "Zepp") ])

,sourceName,sourceVersion,unit,creationDate,startDate,endDate,value,invalid_flag,zero_flag,metric
172579,Zepp,202510161629,count/min,2025-10-26 23:20:58 +0530,2025-10-26 23:19:00+05:30,2025-10-26 23:19:59 +0530,70.0,0,0,heart_rate
172580,Zepp,202510161629,count/min,2025-10-26 23:22:10 +0530,2025-10-26 23:20:00+05:30,2025-10-26 23:20:59 +0530,73.0,0,0,heart_rate
172581,Zepp,202510161629,count/min,2025-10-26 23:22:10 +0530,2025-10-26 23:21:00+05:30,2025-10-26 23:21:59 +0530,73.0,0,0,heart_rate
172582,Zepp,202510161629,count/min,2025-10-26 23:48:57 +0530,2025-10-26 23:22:00+05:30,2025-10-26 23:22:59 +0530,76.0,0,0,heart_rate
172583,Zepp,202510161629,count/min,2025-10-26 23:48:57 +0530,2025-10-26 23:31:00+05:30,2025-10-26 23:31:59 +0530,78.0,0,0,heart_rate
...,...,...,...,...,...,...,...,...,...,...
208250,Zepp,202511201645,count/min,2025-12-03 07:19:37 +0530,2025-12-02 21:36:51+05:30,2025-12-02 21:36:51 +0530,107.0,0,0,heart_rate
208251,Zepp,202511201645,count/min,2025-12-03 07:19:37 +0530,2025-12-02 21:36:56+05:30,2025-12-02 21:36:56 +0530,107.0,0,0,heart_rate
208252,Zepp,202511201645,count/min,2025-12-03 07:19:37 +0530,2025-12-02 21:37:02+05:30,2025-12-02 21:37:02 +0530,107.0,0,0,heart_rate
208253,Zepp,202511201645,count/min,2025-12-03 07:19:37 +0530,2025-12-02 21:37:04+05:30,2025-12-02 21:37:04 +0530,106.0,0,0,heart_rate


In [82]:
hr_clean["sourceName"] = hr_clean["sourceName"].replace({
    "Zepp": "Amazfit",
    "Power Sync": "Fitbit"
})

In [83]:
hr_clean["sourceName"].value_counts()

sourceName
Hiro’s Apple Watch    172636
Amazfit                35452
Fitbit                   166
Fitdays                    1
Name: count, dtype: int64

In [85]:
# Check unique versions per source
(
    hr_clean
    .groupby("sourceName")["sourceVersion"]
    .unique()
)

sourceName
Amazfit               [202510161629, 202510291354, 202511151142, 202...
Fitbit                                                              [1]
Fitdays                                                             [6]
Hiro’s Apple Watch    [11.0.1, 10.3, 11.0, 10.2, 9.0, 11.3.1, 11.5, ...
Name: sourceVersion, dtype: object

In [86]:
(
    hr_clean
    .groupby("sourceName")["sourceVersion"]
    .nunique()
)


sourceName
Amazfit                4
Fitbit                 1
Fitdays                1
Hiro’s Apple Watch    11
Name: sourceVersion, dtype: int64

In [87]:
hr_clean[["sourceName", "sourceVersion"]].head()

,sourceName,sourceVersion
0,Hiro’s Apple Watch,11.0.1
1,Hiro’s Apple Watch,11.0.1
2,Hiro’s Apple Watch,11.0.1
3,Hiro’s Apple Watch,11.0.1
4,Hiro’s Apple Watch,11.0.1


In [88]:
hr_clean.loc[hr_clean["sourceName"] == "Fitdays"]

,sourceName,sourceVersion,unit,creationDate,startDate,endDate,value,invalid_flag,zero_flag,metric
154833,Fitdays,6,count/min,2025-10-04 22:15:15 +0530,2025-10-04 22:15:15+05:30,2025-10-04 22:15:15 +0530,78.0,0,0,heart_rate


In [89]:
hr_clean = hr_clean[hr_clean["sourceName"] != "Fitdays"]

In [90]:
hr_clean["sourceName"].value_counts()

sourceName
Hiro’s Apple Watch    172636
Amazfit                35452
Fitbit                   166
Name: count, dtype: int64

In [91]:
hr_clean.shape

(208254, 10)

In [92]:
hr_clean[["sourceName", "sourceVersion"]].drop_duplicates().sort_values(
    ["sourceName", "sourceVersion"]
)

,sourceName,sourceVersion
172579,Amazfit,202510161629
177881,Amazfit,202510291354
186054,Amazfit,202511151142
203978,Amazfit,202511201645
57839,Fitbit,1
55437,Hiro’s Apple Watch,10.2
20229,Hiro’s Apple Watch,10.3
41401,Hiro’s Apple Watch,11.0
0,Hiro’s Apple Watch,11.0.1
119172,Hiro’s Apple Watch,11.3.1


In [93]:
hr_clean

,sourceName,sourceVersion,unit,creationDate,startDate,endDate,value,invalid_flag,zero_flag,metric
0,Hiro’s Apple Watch,11.0.1,count/min,2024-11-09 05:59:13 +0530,2024-11-09 05:56:05+05:30,2024-11-09 05:56:05 +0530,84.0,0,0,heart_rate
1,Hiro’s Apple Watch,11.0.1,count/min,2024-11-09 06:02:16 +0530,2024-11-09 05:57:47+05:30,2024-11-09 05:57:47 +0530,77.0,0,0,heart_rate
2,Hiro’s Apple Watch,11.0.1,count/min,2024-11-09 06:10:13 +0530,2024-11-09 06:05:50+05:30,2024-11-09 06:05:50 +0530,73.0,0,0,heart_rate
3,Hiro’s Apple Watch,11.0.1,count/min,2024-11-09 06:11:55 +0530,2024-11-09 06:07:50+05:30,2024-11-09 06:07:50 +0530,74.0,0,0,heart_rate
4,Hiro’s Apple Watch,11.0.1,count/min,2024-11-09 06:18:10 +0530,2024-11-09 06:11:35+05:30,2024-11-09 06:11:35 +0530,79.0,0,0,heart_rate
...,...,...,...,...,...,...,...,...,...,...
208250,Amazfit,202511201645,count/min,2025-12-03 07:19:37 +0530,2025-12-02 21:36:51+05:30,2025-12-02 21:36:51 +0530,107.0,0,0,heart_rate
208251,Amazfit,202511201645,count/min,2025-12-03 07:19:37 +0530,2025-12-02 21:36:56+05:30,2025-12-02 21:36:56 +0530,107.0,0,0,heart_rate
208252,Amazfit,202511201645,count/min,2025-12-03 07:19:37 +0530,2025-12-02 21:37:02+05:30,2025-12-02 21:37:02 +0530,107.0,0,0,heart_rate
208253,Amazfit,202511201645,count/min,2025-12-03 07:19:37 +0530,2025-12-02 21:37:04+05:30,2025-12-02 21:37:04 +0530,106.0,0,0,heart_rate


In [95]:
# Ensure datetime
hr_clean.loc[:,"startDate"] = pd.to_datetime(hr_clean["startDate"])

# Create date column
hr_clean.loc[:,"date"] = hr_clean["startDate"].dt.date

In [96]:
hr_clean

,sourceName,sourceVersion,unit,creationDate,startDate,endDate,value,invalid_flag,zero_flag,metric,date
0,Hiro’s Apple Watch,11.0.1,count/min,2024-11-09 05:59:13 +0530,2024-11-09 05:56:05+05:30,2024-11-09 05:56:05 +0530,84.0,0,0,heart_rate,2024-11-09
1,Hiro’s Apple Watch,11.0.1,count/min,2024-11-09 06:02:16 +0530,2024-11-09 05:57:47+05:30,2024-11-09 05:57:47 +0530,77.0,0,0,heart_rate,2024-11-09
2,Hiro’s Apple Watch,11.0.1,count/min,2024-11-09 06:10:13 +0530,2024-11-09 06:05:50+05:30,2024-11-09 06:05:50 +0530,73.0,0,0,heart_rate,2024-11-09
3,Hiro’s Apple Watch,11.0.1,count/min,2024-11-09 06:11:55 +0530,2024-11-09 06:07:50+05:30,2024-11-09 06:07:50 +0530,74.0,0,0,heart_rate,2024-11-09
4,Hiro’s Apple Watch,11.0.1,count/min,2024-11-09 06:18:10 +0530,2024-11-09 06:11:35+05:30,2024-11-09 06:11:35 +0530,79.0,0,0,heart_rate,2024-11-09
...,...,...,...,...,...,...,...,...,...,...,...
208250,Amazfit,202511201645,count/min,2025-12-03 07:19:37 +0530,2025-12-02 21:36:51+05:30,2025-12-02 21:36:51 +0530,107.0,0,0,heart_rate,2025-12-02
208251,Amazfit,202511201645,count/min,2025-12-03 07:19:37 +0530,2025-12-02 21:36:56+05:30,2025-12-02 21:36:56 +0530,107.0,0,0,heart_rate,2025-12-02
208252,Amazfit,202511201645,count/min,2025-12-03 07:19:37 +0530,2025-12-02 21:37:02+05:30,2025-12-02 21:37:02 +0530,107.0,0,0,heart_rate,2025-12-02
208253,Amazfit,202511201645,count/min,2025-12-03 07:19:37 +0530,2025-12-02 21:37:04+05:30,2025-12-02 21:37:04 +0530,106.0,0,0,heart_rate,2025-12-02


In [98]:
HIGH_HR_THRESHOLD = 120   # elevated stress / exertion
LOW_HR_THRESHOLD = 50    # abnormally low for awake state

daily_hr = (
    hr_clean
    .groupby("date")
    .agg(
        avg_hr=("value", "mean"),
        min_hr=("value", "min"),
        max_hr=("value", "max"),
        hr_std=("value", "std"),
        high_hr_minutes=("value", lambda x: (x > HIGH_HR_THRESHOLD).sum()),
        low_hr_minutes=("value", lambda x: (x < LOW_HR_THRESHOLD).sum()),
    )
    .reset_index()
)

In [99]:
daily_hr

,date,avg_hr,min_hr,max_hr,hr_std,high_hr_minutes,low_hr_minutes
0,2022-04-12,71.000000,71.0,71.0,NaN,0,0
1,2022-04-13,69.000000,69.0,69.0,0.000000,0,0
2,2022-04-14,71.000000,71.0,71.0,0.000000,0,0
3,2022-04-15,69.000000,69.0,69.0,0.000000,0,0
4,2022-04-16,68.000000,68.0,68.0,0.000000,0,0
...,...,...,...,...,...,...,...
572,2025-11-18,121.945494,64.0,156.0,18.642647,737,0
573,2025-11-19,104.189804,57.0,143.0,17.002201,727,0
574,2025-11-20,78.280000,53.0,150.0,17.089324,42,0
575,2025-12-02,109.947290,71.0,148.0,13.643768,963,0


In [100]:
daily_hr.describe()

,avg_hr,min_hr,max_hr,hr_std,high_hr_minutes,low_hr_minutes
count,577.000000,577.000000,577.000000,573.000000,577.000000,577.000000
mean,89.151982,54.517375,126.493293,15.852887,86.693241,0.642981
std,15.929782,11.286039,31.677337,8.671995,306.308035,1.342711
min,59.000000,40.000000,59.000000,0.000000,0.000000,0.000000
25%,78.000649,46.000000,116.000000,10.661607,0.000000,0.000000
50%,90.115406,53.000000,133.000000,17.471003,14.000000,0.000000
75%,99.565323,60.000000,146.000000,22.218059,65.000000,1.000000
max,170.000000,167.000000,203.000000,45.027094,4276.000000,19.000000


In [101]:
daily_hr.isna().sum()

date               0
avg_hr             0
min_hr             0
max_hr             0
hr_std             4
high_hr_minutes    0
low_hr_minutes     0
dtype: int64

In [102]:
hr_clean

,sourceName,sourceVersion,unit,creationDate,startDate,endDate,value,invalid_flag,zero_flag,metric,date
0,Hiro’s Apple Watch,11.0.1,count/min,2024-11-09 05:59:13 +0530,2024-11-09 05:56:05+05:30,2024-11-09 05:56:05 +0530,84.0,0,0,heart_rate,2024-11-09
1,Hiro’s Apple Watch,11.0.1,count/min,2024-11-09 06:02:16 +0530,2024-11-09 05:57:47+05:30,2024-11-09 05:57:47 +0530,77.0,0,0,heart_rate,2024-11-09
2,Hiro’s Apple Watch,11.0.1,count/min,2024-11-09 06:10:13 +0530,2024-11-09 06:05:50+05:30,2024-11-09 06:05:50 +0530,73.0,0,0,heart_rate,2024-11-09
3,Hiro’s Apple Watch,11.0.1,count/min,2024-11-09 06:11:55 +0530,2024-11-09 06:07:50+05:30,2024-11-09 06:07:50 +0530,74.0,0,0,heart_rate,2024-11-09
4,Hiro’s Apple Watch,11.0.1,count/min,2024-11-09 06:18:10 +0530,2024-11-09 06:11:35+05:30,2024-11-09 06:11:35 +0530,79.0,0,0,heart_rate,2024-11-09
...,...,...,...,...,...,...,...,...,...,...,...
208250,Amazfit,202511201645,count/min,2025-12-03 07:19:37 +0530,2025-12-02 21:36:51+05:30,2025-12-02 21:36:51 +0530,107.0,0,0,heart_rate,2025-12-02
208251,Amazfit,202511201645,count/min,2025-12-03 07:19:37 +0530,2025-12-02 21:36:56+05:30,2025-12-02 21:36:56 +0530,107.0,0,0,heart_rate,2025-12-02
208252,Amazfit,202511201645,count/min,2025-12-03 07:19:37 +0530,2025-12-02 21:37:02+05:30,2025-12-02 21:37:02 +0530,107.0,0,0,heart_rate,2025-12-02
208253,Amazfit,202511201645,count/min,2025-12-03 07:19:37 +0530,2025-12-02 21:37:04+05:30,2025-12-02 21:37:04 +0530,106.0,0,0,heart_rate,2025-12-02


In [105]:
daily_hr["elevated_hr_flag"] = (daily_hr["avg_hr"] >= 100).astype(int)
#100 bpm daily average is clearly abnormal

In [106]:
daily_hr["high_hr_variability_flag"] = (daily_hr["hr_std"] > 20).astype(int)

In [108]:
daily_hr_export = daily_hr.copy()

In [109]:
daily_hr_export

,date,avg_hr,min_hr,max_hr,hr_std,high_hr_minutes,low_hr_minutes,elevated_hr_flag,high_hr_variability_flag
0,2022-04-12,71.000000,71.0,71.0,NaN,0,0,0,0
1,2022-04-13,69.000000,69.0,69.0,0.000000,0,0,0,0
2,2022-04-14,71.000000,71.0,71.0,0.000000,0,0,0,0
3,2022-04-15,69.000000,69.0,69.0,0.000000,0,0,0,0
4,2022-04-16,68.000000,68.0,68.0,0.000000,0,0,0,0
...,...,...,...,...,...,...,...,...,...
572,2025-11-18,121.945494,64.0,156.0,18.642647,737,0,1,0
573,2025-11-19,104.189804,57.0,143.0,17.002201,727,0,1,0
574,2025-11-20,78.280000,53.0,150.0,17.089324,42,0,0,0
575,2025-12-02,109.947290,71.0,148.0,13.643768,963,0,1,0


In [110]:
daily_hr_export.to_csv(
    "daily_heart_rate_analytics.csv",
    index=False
)